### Baseline model (Plus)에 있는 기능

- binary_load_dataset 메서드

데이터 비율 조정을 위한 파라미터 값을 기반으로 데이터를 로드하는 메서드로

주어진 binary_classification_data.csv 파일을 열고 데이터를 읽습니다. 데이터의 마지막 열에 종속 변수가 있으며, 해당 열의 값에 따라서 '1'인 pulsar 데이터와 '0'인 star 데이터를 각각 별도의 리스트에 저장합니다.

그 후, 실험에 사용될 데이터와 신경망의 입력 및 출력 크기를 전역 변수로 선언합니다. pulsar와 star 데이터는 하나의 변수 data에 그룹화됩니다. pulsar 데이터의 수를 star 데이터의 수에 맞추기 위해 데이터의 수를 조정할 수 있습니다.

adjust_ratio 매개변수가 True인 경우, pulsar 데이터의 비율을 증가시킵니다. pulsar 데이터와 star 데이터를 하나의 변수에 할당하고, 데이터의 크기는 star 데이터의 배수로 저장됩니다.

adjust_ratio 값이 True인 경우 star 데이터를 data 변수에 할당한 후, pulsar 데이터를 star 데이터의 수만큼 반복하여 추가합니다. 이를 위해 % 연산자를 사용하여 pulsar 데이터의 인덱스를 순차적으로 추출합니다.

adjust_ratio 값이 False인 경우, 기존 데이터를 그대로 data 변수에 할당합니다. star 데이터를 먼저 할당하고, 그 다음에 pulsar 데이터를 할당합니다.

### 데이터 불러오기

In [3]:
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.layers import BatchNormalization
from keras.layers import Dropout
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

### 데이터 불균형 처리
- 데이터 처리

1. oversampling :

   SMOTE, ADASYN

2. undersampling

- 평가지표 처리

1. accurcacy :

   대부분의 예측이 다수 클래스에 속하게 되어 별로

2. F1-score :

   Baseline model에서 사용한 평가지표로 불균형한 데이터에서 평가지표로 사용

- 모델 구현시 처리

1. 클래스 가중치 설정 :

   소수 클래스에 더 큰 가중치 부여


In [36]:
# 새로운 데이터 (EDA)과정 진행 후 데이터
X_train_smote = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/X_train_MS_smote.csv')
X_train_adasyn = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/X_train_MS_adasyn.csv')
X_test = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/X_test_MS.csv')
X_val = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/X_val_MS.csv')
y_train_smote = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/y_train_MS_smote.csv')
y_train_adasyn = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/y_train_MS_adasyn.csv')
y_test = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/y_test.csv')
y_val = pd.read_csv('csv/Binary_to_share/Binary_to_share/Minmax+Standard/y_val.csv')

In [37]:
y_train_smote = y_train_smote['target_class']
y_test = y_test['target_class']
y_val = y_val['target_class']
# 클래스 가중치 계산
class_weight1 = {0: 1., 1: (len(y_train_smote) / sum(y_train_smote))}

### 모델 구현 
- 최적의 파라미터를 반복문을 통해 찾기

- 이진 분류 모델에서는 대표적으로 Adam optimizer를 사용하는 것이 일반적입니다. 다른 optimizer로는 SGD, RMSprop 등이 있으며, 이들 중에서도 데이터셋의 크기와 모델의 복잡도에 따라 적합한 optimizer를 선택해야 합니다. 하지만 대체로 Adam optimizer가 다른 optimizer보다 성능이 좋은 편입니다.

- 모델에 배치 정규화 레이어를 추가하면 모델이 더 안정적으로 수렴할 수 있습니다. 

In [7]:
# 복습한 내용으로 추가적인 진행 - adam 사용, adasyn
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt


# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst =[32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}

train_lst = [X_train_adasyn, X_train_smote]

best_metric = 0.0
metric_hyper = {}


# dropout, 배치 정규화 추가
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_adasyn.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

            # 모델 학습
           
            model.fit(X_train_adasyn, y_train_adasyn, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight2)
            tmp = class_weight2
            

            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
613/613 [==============================] - 7s 5ms/step - loss: 1.0679 - accuracy: 0.7745 - val_loss: 1.6730 - val_accuracy: 0.6713
Epoch 2/1000
613/613 [==============================] - 3s 4ms/step - loss: 0.8634 - accuracy: 0.7813 - val_loss: 0.8466 - val_accuracy: 0.8294
Epoch 3/1000
613/613 [==============================] - 3s 6ms/step - loss: 0.6813 - accuracy: 0.7906 - val_loss: 0.6487 - val_accuracy: 0.6993
Epoch 4/1000
613/613 [==============================] - 3s 5ms/step - loss: 0.6740 - accuracy: 0.7949 - val_loss: 0.6209 - val_accuracy: 0.7276
Epoch 5/1000
613/613 [==============================] - 3s 5ms/step - loss: 0.6304 - accuracy: 0.8076 - val_loss: 1.0789 - val_accuracy: 0.6761
Epoch 6/1000
613/613 [==============================] - 3s 5ms/step - loss: 0.7058 - accuracy: 0.8124 - val_loss: 0.8395 - val_accuracy: 0.6934
Epoch 7/1000
613/613 [==============================] - 3s 5ms/step - loss: 0.7134 - accuracy: 0.8106 - val_loss: 0.5996 - val_accuracy:

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 249ms/epoch - 3ms/step
Epoch 1/1000
307/307 [==============================] - 9s 8ms/step - loss: 1.0490 - accuracy: 0.7682 - val_loss: 0.6474 - val_accuracy: 0.7551
Epoch 2/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.8093 - accuracy: 0.7922 - val_loss: 1.4289 - val_accuracy: 0.7923
Epoch 3/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.8078 - accuracy: 0.7836 - val_loss: 0.5964 - val_accuracy: 0.8279
Epoch 4/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.7371 - accuracy: 0.7941 - val_loss: 0.7191 - val_accuracy: 0.8401
Epoch 5/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.6799 - accuracy: 0.8045 - val_loss: 0.6175 - val_accuracy: 0.8195
Epoch 6/1000
307/307 [==============================] - 2s 5ms/step - loss: 0.6579 - accuracy: 0.8124 - val_loss: 0.6865 - val_accuracy: 0.8040
Epoch 7/1000
307/307 [==============================] - 2s 6ms/ste

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 187ms/epoch - 2ms/step
Epoch 1/1000
307/307 [==============================] - 9s 9ms/step - loss: 1.8818 - accuracy: 0.7121 - val_loss: 2.4096 - val_accuracy: 0.6901
Epoch 2/1000
307/307 [==============================] - 2s 7ms/step - loss: 1.2022 - accuracy: 0.7317 - val_loss: 1.7968 - val_accuracy: 0.7265
Epoch 3/1000
307/307 [==============================] - 2s 6ms/step - loss: 1.1542 - accuracy: 0.7117 - val_loss: 0.5357 - val_accuracy: 0.8680
Epoch 4/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.6998 - accuracy: 0.7699 - val_loss: 0.7604 - val_accuracy: 0.7272
Epoch 5/1000
307/307 [==============================] - 2s 6ms/step - loss: 0.6748 - accuracy: 0.7835 - val_loss: 0.9949 - val_accuracy: 0.7566
Epoch 6/1000
307/307 [==============================] - 2s 8ms/step - loss: 0.7709 - accuracy: 0.6739 - val_loss: 0.8596 - val_accuracy: 0.0960
Epoch 7/1000
307/307 [==============================] - 2s 8ms/ste

In [38]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt


# 변수 리스트 생성
act_func = ['relu', 'tanh', 'sigmoid', 'elu']
batch_lst =[32, 64, 128, 256]
best_accuracy = 0.0
best_hyperparams = {}
lr_lst = [0.009, 0.006, 0.003, 0.001, 0.0005]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# dropout, 배치 정규화 추가
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(16, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation=func))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1, verbose = 0)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 117ms/epoch - 1ms/step


c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.8884 - accuracy: 0.9401 - 194ms/epoch - 2ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 1.0606 - accuracy: 0.9268 - 130ms/epoch - 2ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.8344 - accuracy: 0.9430 - 126ms/epoch - 1ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 118ms/epoch - 1ms/step


c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 127ms/epoch - 1ms/step


...

85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.2312 - accuracy: 0.9518 - 132ms/epoch - 2ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.3193 - accuracy: 0.9471 - 140ms/epoch - 2ms/step
85/85 [==============================] - 0s 1000us/step
85/85 - 0s - loss: 0.3637 - accuracy: 0.9695 - 113ms/epoch - 1ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.2946 - accuracy: 0.9540 - 116ms/epoch - 1ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.3638 - accuracy: 0.9349 - 166ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.1766 - accuracy: 0.9691 - 139ms/epoch - 2ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.2254 - accuracy: 0.9533 - 129ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.2536 - accuracy: 0.9577 - 141ms/epoch - 2ms/step
85/85 [======

c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 [==============================] - 0s 2ms/step


c:\Users\mj985\anaconda3\envs\assign\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


85/85 - 0s - loss: 1.4801 - accuracy: 0.9040 - 183ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.2588 - accuracy: 0.9401 - 167ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 1.9862 - accuracy: 0.8699 - 149ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 3.3704 - accuracy: 0.7790 - 161ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.9213 - accuracy: 0.9397 - 148ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.2899 - accuracy: 0.9592 - 147ms/epoch - 2ms/step
85/85 [==============================] - 0s 1ms/step
85/85 - 0s - loss: 0.2309 - accuracy: 0.9651 - 123ms/epoch - 1ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - loss: 0.2885 - accuracy: 0.9614 - 139ms/epoch - 2ms/step
85/85 [==============================] - 0s 2ms/step
85/85 - 0s - los

In [9]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 위에서 얻은 정보로 parameter 수 줄이기 -> 속도 향상

# 변수 리스트 생성
act_func = ['relu', 'elu']
best_accuracy = 0.0
batch_lst = [32, 64, 128, 256]
best_hyperparams = {}
lr_lst = [0.006, 0.003, 0.001]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# dropout, 배치 정규화 추가
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(128, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(64, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(8, activation=func))
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
615/615 [==============================] - 4s 3ms/step - loss: 0.2710 - accuracy: 0.9316 - val_loss: 0.1392 - val_accuracy: 0.9585
Epoch 2/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2378 - accuracy: 0.9392 - val_loss: 0.2892 - val_accuracy: 0.9364
Epoch 3/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2279 - accuracy: 0.9438 - val_loss: 0.1792 - val_accuracy: 0.9563
Epoch 4/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2174 - accuracy: 0.9444 - val_loss: 0.2251 - val_accuracy: 0.9228
Epoch 5/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2144 - accuracy: 0.9460 - val_loss: 0.2322 - val_accuracy: 0.9199
Epoch 6/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2131 - accuracy: 0.9443 - val_loss: 0.2074 - val_accuracy: 0.9338
Epoch 7/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2077 - accuracy: 0.9475 - val_loss: 0.1547 - val_accuracy:

In [10]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 위에서 얻은 정보로 parameter 수 줄이기 -> 속도 향상

# 변수 리스트 생성
act_func = ['relu', 'elu']
best_accuracy = 0.0
batch_lst = [32, 64, 128, 256]
best_hyperparams = {}
lr_lst = [0.006, 0.003, 0.001]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# 다이아몬드 형 체크
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가
            model.add(Dense(512, activation=func))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가              
            model.add(Dense(128, activation=func))
            model.add(Dense(32, activation=func))
            model.add(Dense(4, activation=func))
            model.add(Dense(4, activation=func))
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
615/615 [==============================] - 4s 4ms/step - loss: 0.2725 - accuracy: 0.9290 - val_loss: 0.2178 - val_accuracy: 0.9574
Epoch 2/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.2408 - accuracy: 0.9430 - val_loss: 0.1765 - val_accuracy: 0.9559
Epoch 3/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.2294 - accuracy: 0.9428 - val_loss: 0.2001 - val_accuracy: 0.9551
Epoch 4/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.2221 - accuracy: 0.9445 - val_loss: 0.2503 - val_accuracy: 0.8934
Epoch 5/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.2162 - accuracy: 0.9453 - val_loss: 0.1753 - val_accuracy: 0.9507
Epoch 6/1000
615/615 [==============================] - 2s 4ms/step - loss: 0.2053 - accuracy: 0.9475 - val_loss: 0.1477 - val_accuracy: 0.9537
Epoch 7/1000
615/615 [==============================] - 2s 3ms/step - loss: 0.2058 - accuracy: 0.9482 - val_loss: 0.1763 - val_accuracy:

In [11]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 위에서 얻은 정보로 parameter 수 줄이기 -> 속도 향상

# 변수 리스트 생성
act_func = ['relu', 'elu']
best_accuracy = 0.0
batch_lst = [32, 64, 128, 256]
best_hyperparams = {}
lr_lst = [0.006, 0.003, 0.001]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# 다이아몬드 형 체크
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
            model.add(BatchNormalization()) # 배치 정규화 추가
            model.add(Dropout(0.2))  # Dropout 추가             
            model.add(Dense(32, activation=func))
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
615/615 [==============================] - 2s 2ms/step - loss: 0.2587 - accuracy: 0.9358 - val_loss: 0.1854 - val_accuracy: 0.9489
Epoch 2/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2236 - accuracy: 0.9463 - val_loss: 0.1318 - val_accuracy: 0.9676
Epoch 3/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2168 - accuracy: 0.9459 - val_loss: 0.2281 - val_accuracy: 0.9537
Epoch 4/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2094 - accuracy: 0.9470 - val_loss: 0.2109 - val_accuracy: 0.9404
Epoch 5/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2079 - accuracy: 0.9464 - val_loss: 0.2132 - val_accuracy: 0.9419
Epoch 6/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2046 - accuracy: 0.9478 - val_loss: 0.1479 - val_accuracy: 0.9537
Epoch 7/1000
615/615 [==============================] - 1s 2ms/step - loss: 0.2009 - accuracy: 0.9465 - val_loss: 0.1658 - val_accuracy:

In [12]:
# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 위에서 얻은 정보로 parameter 수 줄이기 -> 속도 향상

# 변수 리스트 생성
act_func = ['relu', 'elu']
best_accuracy = 0.0
batch_lst = [32, 64, 128, 256]
best_hyperparams = {}
lr_lst = [0.006, 0.003, 0.001]
best_time = 11111.0
time_hyper = {}


best_metric = 0.0
metric_hyper = {}


# 다이아몬드 형 체크
# 모델 구현
for func in act_func:
    for i in lr_lst:
        for batch in batch_lst:
            metrics_lst = []

            # model구현
            model = Sequential()
            model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))           
            model.add(Dense(32, activation=func))
            model.add(Dense(1, activation='sigmoid'))

            # 모델 컴파일
            model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

            # Early stopping 기능 추가
            early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
            start_time = time.time()

        
            model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1)
            tmp = class_weight1
         


            end_time = time.time()
            long_time = end_time - start_time
            if long_time < best_time:
                best_time = long_time
                time_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}


            # 모델 평가
            y_pred = model.predict(X_val)
            y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
            metrics_lst.append(f1_score(y_val, y_pred_binary))
            metrics_lst.append(precision_score(y_val, y_pred_binary))
            metrics_lst.append(recall_score(y_val, y_pred_binary))
            mean_met = np.mean(metrics_lst)

            if mean_met > best_metric:
                best_metric = mean_met
                metric_hyper = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

            loss, acc = model.evaluate(X_val, y_val, verbose = 2)

            if acc > best_accuracy:
                best_accuracy = acc
                best_hyperparams = {'activation': func, 'learning_rate': i, 'X_train': tmp, 'batch': batch}

print('Best hyperparameters:', best_hyperparams)
print('Best validation accuracy:', best_accuracy)
print('Best time:', best_time)
print('Best time_hyperparams:', time_hyper)
print('Best metric:', best_metric)
print('Best metric_hyperparams:', metric_hyper)


Epoch 1/1000
615/615 [==============================] - 2s 2ms/step - loss: 0.2335 - accuracy: 0.9426 - val_loss: 0.1914 - val_accuracy: 0.9566
Epoch 2/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.2047 - accuracy: 0.9492 - val_loss: 0.2093 - val_accuracy: 0.9493
Epoch 3/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.1923 - accuracy: 0.9516 - val_loss: 0.2169 - val_accuracy: 0.9176
Epoch 4/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.1864 - accuracy: 0.9511 - val_loss: 0.1880 - val_accuracy: 0.9456
Epoch 5/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.1796 - accuracy: 0.9526 - val_loss: 0.1877 - val_accuracy: 0.9368
Epoch 6/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.1744 - accuracy: 0.9520 - val_loss: 0.1871 - val_accuracy: 0.9371
Epoch 7/1000
615/615 [==============================] - 1s 1ms/step - loss: 0.1690 - accuracy: 0.9530 - val_loss: 0.2015 - val_accuracy:

### 최종

In [39]:
# Best hyperparameters: {'activation': 'relu', 'learning_rate': 0.003, 'X_train': {0: 1.0, 1: 2.0}, 'batch': 64}
# Best validation accuracy: 0.9753676652908325
# Best time: 8.609997987747192
# Best time_hyperparams: {'activation': 'elu', 'learning_rate': 0.009, 'X_train': {0: 1.0, 1: 2.0}, 'batch': 256}
# Best metric: 0.8658478183402076
# Best metric_hyperparams: {'activation': 'relu', 'learning_rate': 0.003, 'X_train': {0: 1.0, 1: 2.0}, 'batch': 64}

# 복습한 내용으로 추가적인 진행 - adam 사용, smote
def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt

# 변수 리스트 생성
func = 'relu'
batch = 64
i = 0.003

# model 구현
model = Sequential()
model.add(Dense(256, activation=func, input_shape=(X_train_smote.shape[1],)))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(128, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가              
model.add(Dense(64, activation=func))
model.add(Dense(32, activation=func))
model.add(Dense(16, activation=func))
model.add(Dense(8, activation=func))
model.add(Dense(8, activation=func))
model.add(Dense(1, activation='sigmoid')) # 출력층을 sigmoid , 또는 relu사용 가능


# 모델 컴파일
model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping 기능 추가
early_stopping = EarlyStopping(patience=10, monitor='val_accuracy')
start_time = time.time()

# 모델 적용
model.fit(X_train_smote, y_train_smote, epochs=1000, batch_size=batch, validation_data=(X_val, y_val), callbacks=[early_stopping], class_weight=class_weight1, verbose = 0)

end_time = time.time()
long_time = end_time - start_time



# 모델 평가
y_pred = model.predict(X_train_smote)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
metrics_lst.append(f1_score(y_train_smote, y_pred_binary))
metrics_lst.append(precision_score(y_train_smote, y_pred_binary))
metrics_lst.append(recall_score(y_train_smote, y_pred_binary))
mean_met = np.mean(metrics_lst)

print("train_loss, train_accuracy")
train_loss, train_cc = model.evaluate(X_train_smote, y_train_smote, verbose = 2)

print("val_loss, val_accuracy")
val_loss, val_acc = model.evaluate(X_val, y_val, verbose = 2)

print("걸린시간 :", long_time)

print("=== train set ===")
print('f1_score:', f1_score(y_train_smote, y_pred_binary))
print('precision_score:', precision_score(y_train_smote, y_pred_binary))
print('recall_score', recall_score(y_train_smote, y_pred_binary))

y_pred = model.predict(X_test)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
print("=== test set ===")
print('f1_score:', f1_score(y_test, y_pred_binary))
print('precision_score:', precision_score(y_test, y_pred_binary))
print('recall_score', recall_score(y_test, y_pred_binary))


615/615 [==============================] - 1s 1ms/step
train_loss, train_accuracy
615/615 - 1s - loss: 0.1052 - accuracy: 0.9592 - 671ms/epoch - 1ms/step
val_loss, val_accuracy
85/85 - 0s - loss: 0.1836 - accuracy: 0.9397 - 108ms/epoch - 1ms/step
걸린시간 : 24.028000593185425
=== train set ===
f1_score: 0.9597994987468672
precision_score: 0.94642151047845
recall_score 0.9735611145007118
107/107 [==============================] - 0s 1ms/step
=== test set ===
f1_score: 0.759142496847415
precision_score: 0.6445396145610278
recall_score 0.9233128834355828


## 최종 모델 : 복습 진행 후
- Best hyperparameters: {'activation': 'relu', 'learning_rate': 0.009, 'dropout': 'yes', 'batch_normal': 'yes'}
- Best validation accuracy: 0.9786764979362488
- Best time : 5.906041860580444
- time hyperparameters : {'activation': 'relu', 'learning_rate': 0.003, 'dropout': 'yes', 'batch_normal': 'yes'}

In [21]:
# 복습한 내용으로 추가적인 진행

def custom_opt(n):
    opt = tf.keras.optimizers.Adam(learning_rate=n)
    return opt
func = 'relu'
i = 0.009

# 모델 구현
model = Sequential()
model.add(Dense(256, activation=func, input_shape=(X_train.shape[1],)))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가
model.add(Dense(128, activation=func))
model.add(BatchNormalization()) # 배치 정규화 추가
model.add(Dropout(0.2))  # Dropout 추가              
model.add(Dense(64, activation=func))
model.add(Dense(32, activation=func))
model.add(Dense(16, activation=func))
model.add(Dense(8, activation=func))
model.add(Dense(8, activation=func))
model.add(Dense(1, activation=func))

# 모델 컴파일
model.compile(optimizer=custom_opt(i), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping 기능 추가
early_stopping = EarlyStopping(patience=10, monitor='val_loss')
start_time = time.time()

# 모델 학습
model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[early_stopping])
end_time = time.time()
long_time = end_time - start_time


# 모델 평가
y_pred = model.predict(X_val)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
metrics_lst = []
metrics_lst.append(f1_score(y_val, y_pred_binary))
metrics_lst.append(precision_score(y_val, y_pred_binary))
metrics_lst.append(recall_score(y_val, y_pred_binary))
f1 = f1_score(y_val, y_pred_binary)
roc_auc = roc_auc_score(y_val, y_pred)
precision = precision_score(y_val, y_pred_binary)
recall = recall_score(y_val, y_pred_binary)

print("train loss, acc")
train_loss, train_acc = model.evaluate(X_train_smote, y_train_smote, verbose = 2)

print("validation loss, acc")
loss, acc = model.evaluate(X_val, y_val, verbose = 2)

print("걸린 시간 :",long_time)
print("f1 :", f1)
print("roc_auc :", roc_auc)
print("precision :", precision)
print("recall :", recall)

print(np.mean(metrics_lst))


NameError: name 'X_train' is not defined

### BEST MODEL

1. 구성 추가 - undersampling or oversampling 여부 

2. 최적의 파라미터 적용

3. 평가지표 어떤것을 사용할지

4. 클래스 가중치를 적용할 것인지.

### 추가사항

1. 과연 accuracy가 기존 모델보다 낮아진 모델을 사용해도 될까?: 
 accuracy를 기준으로 early stopping하면 모델의 성능은 더 증가할 것.

2. 좋은 모델의 파라미터를 사용해서 모델 구조를 변경해보기.


In [15]:
print('Best hyperparameters:', best_hyperparams)
print('Best accuracy:', best_accuracy)

Best hyperparameters: {'activation': 'tanh', 'batch_size': 64, 'optimizer': 'sgd', 'dropout': 'yes', 'batch_normal': 'yes'}
Best accuracy: 0.8460195660591125


# 사용

In [ ]:
# KFold를 통해 모델 성능을 평가해보기.
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
precision_scores = []
recalls = []
losses = []
accs = []
test_losses = []
test_accs = []

for train_index, val_index in kf.split(X_train_smote):
    X_train_fold, X_val_fold = X_train_smote[train_index], X_train_smote[val_index]
    y_train_fold, y_val_fold = y_train_smote[train_index], y_train_smote[val_index]
    
    # 모델 생성 및 학습
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train_smote.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early stopping 기능 추가
    early_stopping = EarlyStopping(patience=5, monitor='accuracy')

    model.fit(X_train_fold, y_train_fold, epochs=1000, batch_size=32, class_weight=class_weight1, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])
    
    loss, acc = model.evaluate(X_train_fold, y_train_fold, verbose = 2)
    test_loss, test_acc = model.evaluate(X_val_fold, y_val_fold, verbose = 2)

    losses.append(loss)
    accs.append(acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val_fold)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    
    # f1 score와 precision 계산
    f1_scores.append(f1_score(y_val_fold, y_pred))
    recalls.append(recall_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    
# 교차 검증 결과 출력
print("======== Final Model ========")
print('Mean train loss:', round(np.mean(losses),4))
print('Mean train accuracy:', round(np.mean(accs),4))
print('Mean validation loss:', round(np.mean(test_losses),4))
print('Mean validation accuracy:', round(np.mean(test_accs),4))
print('Mean F1 score:', round(np.mean(f1_scores),4))
print('Mean precision:', round(np.mean(precision_scores),4))
print("Mean recall:", round(np.mean(recalls),4))

In [25]:
# KFold를 통해 모델 성능을 평가해보기.
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []
precision_scores = []
recalls = []
losses = []
accs = []
test_losses = []
test_accs = []

for train_index, val_index in kf.split(X_train_smote):
    X_train_fold, X_val_fold = X_train_smote[train_index], X_train_smote[val_index]
    y_train_fold, y_val_fold = y_train_smote[train_index], y_train_smote[val_index]
    
    # 모델 생성 및 학습
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train_smote.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early stopping 기능 추가
    early_stopping = EarlyStopping(patience=5, monitor='accuracy')

    model.fit(X_train_fold, y_train_fold, epochs=1000, batch_size=32, class_weight=class_weight, validation_data=(X_val_fold, y_val_fold), callbacks=[early_stopping])
    
    loss, acc = model.evaluate(X_train_fold, y_train_fold, verbose = 2)
    test_loss, test_acc = model.evaluate(X_val_fold, y_val_fold, verbose = 2)

    losses.append(loss)
    accs.append(acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    # 검증 데이터에 대한 예측 수행
    y_pred = model.predict(X_val_fold)
    y_pred = [1 if pred > 0.5 else 0 for pred in y_pred]
    
    # f1 score와 precision 계산
    f1_scores.append(f1_score(y_val_fold, y_pred))
    recalls.append(recall_score(y_val_fold, y_pred))
    precision_scores.append(precision_score(y_val_fold, y_pred))
    
# 교차 검증 결과 출력
print("======== Final Model ========")
print('Mean train loss:', round(np.mean(losses),4))
print('Mean train accuracy:', round(np.mean(accs),4))
print('Mean validation loss:', round(np.mean(test_losses),4))
print('Mean validation accuracy:', round(np.mean(test_accs),4))
print('Mean F1 score:', round(np.mean(f1_scores),4))
print('Mean precision:', round(np.mean(precision_scores),4))
print("Mean recall:", round(np.mean(recalls),4))

Epoch 1/1000
521/521 [==============================] - 1s 2ms/step - loss: 0.7848 - accuracy: 0.8082 - val_loss: 0.3228 - val_accuracy: 0.8435
Epoch 2/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.5158 - accuracy: 0.8549 - val_loss: 0.2591 - val_accuracy: 0.8818
Epoch 3/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4589 - accuracy: 0.8793 - val_loss: 0.2802 - val_accuracy: 0.8791
Epoch 4/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4380 - accuracy: 0.8879 - val_loss: 0.2921 - val_accuracy: 0.8753
Epoch 5/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4180 - accuracy: 0.8930 - val_loss: 0.3105 - val_accuracy: 0.8661
Epoch 6/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4109 - accuracy: 0.8935 - val_loss: 0.2440 - val_accuracy: 0.8981
Epoch 7/1000
521/521 [==============================] - 1s 1ms/step - loss: 0.4014 - accuracy: 0.8975 - val_loss: 0.2137 - val_accuracy: